In [1]:
import pandas as pd
import sklearn
import pickle
from get_words_by_Natasha import get_words

<function get_words at 0x000001BDD5CAC860>


In [2]:
from freq_of_symbol import freq_of_symbol
from freq_of_word import freq_of_word
from number_of_unique_words import number_of_unique_words
from number_of_words_that_occur_once import number_of_words_that_occur_once


In [3]:
def dataset_to_pandas_table(dataset = pickle.load(open("dataset.pkl", "rb"))):
    authors = list(set(dataset['authors']))
    dict_authors = dict()
    reverse_dict_authors = dict()
    for index in range(len(authors)):
        dict_authors[authors[index]] = index
        reverse_dict_authors[index] = authors[index]

    target = []
    for author in dataset["authors"]:
        target += [dict_authors[author]]
    feature1 = []
    feature2 = []
    for book in dataset['books']:
        words = get_words(book.lower())
        #сюда лемматизацию впихуем

        #сюда по очереди все функции
        feature1.append(freq_of_word(words, 'и'))
    dataset_for_ml = {"feature1": feature1, "targets": target}
    df = pd.DataFrame(dataset_for_ml)
    return df

In [5]:
df = dataset_to_pandas_table()
authors_seen = []
test_ixs = []
train_ixs = []
for ixs in range(len(df)):
    aut = df['targets'][ixs]
    if authors_seen.count(aut) < 2:
        authors_seen += [aut]
        test_ixs += [ixs]
    else: train_ixs += [ixs]

y_test = df.iloc[test_ixs]['targets']
x_test = df.iloc[test_ixs].drop(columns=["targets"])
y_train = df.iloc[train_ixs]['targets']
x_train = df.iloc[train_ixs].drop(columns=["targets"])

In [6]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
rfc = ExtraTreesClassifier()
rfc.fit(x_train, y_train)
accuracy_score(y_test, rfc.predict(x_test))

0.07

In [ ]:
rfc.predict(x_test)